<h1> Python code for scraping one week of Weather Underground data, collecting once per hour, and saving into a file in the directory wunderground_data </h1>

<h3> Import necessary Python libraries </h3>

In [62]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from bs4 import BeautifulSoup
import urllib
import json
import requests
import datetime
# import dicttoxml
%matplotlib inline

<h3> Set variables/initialize data structures once per week at top of hour: </h3>

In [176]:
dictionary = {}

time = datetime.datetime.now()
start_time = time

# week_str = datetime.datetime.strftime(time, "%W")
# week_int = int(week_str)

collect_time_str = datetime.datetime.strftime(time, "%m-%d %H-%M")
collect_hr_int = int(collect_time_str[-2:])

# file = open('wunderground_data/' + collect_time_str + '.json', 'w')

# read initial wunderground json page

# data = get_json()
# add_to_dict(data)


<h3> Define functions: </h3>

In [170]:
def update_time():
    time = datetime.datetime.now()
    return time
    
def update_collect_time():
    collect_time_str = datetime.datetime.strftime(time, "%m-%d %H-%M")
    collect_hr_int = int(collect_time_str[-5:-3])
    return [collect_time_str, collect_hr_int]

def get_json():
    page = requests.get('http://api.wunderground.com/api/f71059696f42912d/forecast/geolookup/conditions/q/MA/Boston.json')
    data = json.loads(page.content)
    return data

def add_to_dict(data):
    key = str(collect_time_str)
    dictionary[key] = data

def write_file(dictionary):
    # writing into file
    json.dump(dictionary, file)
    
def get_timedelta_mins(delta):
    return (delta.seconds/3600)

def get_datetime_min(time):
    return int(datetime.datetime.strftime(time, "%M"))


<h3> Main code for one week of data -- without datetime -- NOT FINISHED: </h3>

In [ ]:
hr = int(datetime.datetime.strftime(time, "%H"))
day = 1
collections = 0
week_length = 7
hrs_in_week = 168
time_not_expired = True

prev_hr = hr
while time_not_expired:
    temp_time = datetime.datetime.now()
    temp_hr = int(datetime.datetime.strftime(temp_time, "%H"))
    temp_day = int(datetime.datetime.strftime(temp_time, "%d"))
    if prev_hr != temp_hr:
        time = update_time()
        hold = update_collect_time()[0]
        collect_time_str = hold[0]
        collect_hr_int = hold[1]
        data = get_json()
        collections += 1
        print('got json' + collect_time_str)
        add_to_dict(data)
        if hr = 24:
            day += 1
    time_not_expired = (day != week_length) and collections == hrs_in_week
    prev_min = temp_min

# write_file(dictionary)
json.dump(dictionary, file)

print('done')

<h3> Main code for one week of data -- with datetime -- NOT FINISHED: </h3>

In [ ]:
# main code for one week of data -- with datetime -- NOT FINISHED

week_length = 7
time_not_expired = True

prev_time = time
while time_not_expired:
    temp_time = datetime.datetime.now()
    hr_delta = temp_time - prev_time
    if hr_delta.hours >= 1:
        time = update_time()
        hold = update_collect_time()[0]
        collect_time_str = hold[0]
        collect_hr_int = hold[1]
        data = get_json()
        print('got json' + collect_time_str)
        add_to_dict(data)
    week_delta = temp_time - start_time
    time_not_expired = (week_delta.days == week_length)
    prev_time = temp_time

# write_file(dictionary)
json.dump(dictionary, file)

print('done')

<h3> 5 minute json file test code -- without datetime: </h3>

In [110]:
# 5 minute test

file = open('wunderground_data_test/fifth_5min_test', 'w')

min = int(datetime.datetime.strftime(time, "%M"))
end_time = 3
time_not_expired = True

prev_min = min
while time_not_expired:
    temp_time = datetime.datetime.now()
    temp_min = int(datetime.datetime.strftime(temp_time, "%M"))
    if prev_min != temp_min:
        time = update_time()
        hold = update_collect_time()[0]
        collect_time_str = hold[0]
        collect_hr_int = hold[1]
        data = get_json()
        print('got json')
        add_to_dict(data)
    time_not_expired = ((temp_min - min) != end_time)
    prev_min = temp_min

# write_file(dictionary)
json.dump(dictionary, file)

print('done')

got json
got json
got json
done


<h3> 5 minute json file test code -- with datetime -- NOT FINISHED: </h3>

In [177]:
test_length = 5
time_not_expired = True

file = open('wunderground_data_test/fifth_5min_test', 'w')

prev_time = time
while time_not_expired:
    temp_time = datetime.datetime.now()
    if get_datetime_min(temp_time) != get_datetime_min(prev_time):
        time = update_time()
        hold = update_collect_time()
        collect_time_str = hold[0]
        collect_hr_int = hold[1]
        data = get_json()
        print('got json' + collect_time_str)
        add_to_dict(data)
    time_passed = temp_time - start_time
    time_not_expired = (get_timedelta_mins(time_passed) != test_length)
    prev_time = temp_time

# write_file(dictionary)
json.dump(dictionary, file)

print('done')

got json06-15 12-58
got json06-15 12-59
got json06-15 13-00
got json06-15 13-01
got json06-15 13-02


ConnectionError: HTTPConnectionPool(host='api.wunderground.com', port=80): Max retries exceeded with url: /api/f71059696f42912d/forecast/geolookup/conditions/q/MA/Boston.json (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x1119a7a90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

<h3> Miscellaneous scraps of code: </h3>

In [108]:
file.close()

In [32]:
#for k, v in dictionary.items():
    #print(k, v)

print(dictionary.keys())
dictionary['current_observation'].keys()

dict_keys(['response', 'location', 'current_observation', 'forecast'])


dict_keys(['image', 'display_location', 'observation_location', 'estimated', 'station_id', 'observation_time', 'observation_time_rfc822', 'observation_epoch', 'local_time_rfc822', 'local_epoch', 'local_tz_short', 'local_tz_long', 'local_tz_offset', 'weather', 'temperature_string', 'temp_f', 'temp_c', 'relative_humidity', 'wind_string', 'wind_dir', 'wind_degrees', 'wind_mph', 'wind_gust_mph', 'wind_kph', 'wind_gust_kph', 'pressure_mb', 'pressure_in', 'pressure_trend', 'dewpoint_string', 'dewpoint_f', 'dewpoint_c', 'heat_index_string', 'heat_index_f', 'heat_index_c', 'windchill_string', 'windchill_f', 'windchill_c', 'feelslike_string', 'feelslike_f', 'feelslike_c', 'visibility_mi', 'visibility_km', 'solarradiation', 'UV', 'precip_1hr_string', 'precip_1hr_in', 'precip_1hr_metric', 'precip_today_string', 'precip_today_in', 'precip_today_metric', 'icon', 'icon_url', 'forecast_url', 'history_url', 'ob_url', 'nowcast'])